### Start SparkSession

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = SparkSession \
            .builder \
            .appName("halltape_pyspark_local") \
            .master("spark://spark-master:7077") \
    .getOrCreate()


print("Активные Spark сессии:", spark.sparkContext.uiWebUrl)

25/07/25 06:29:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Активные Spark сессии: http://b26e130c294d:4040


### Read

In [2]:
PATH = 'data/customs_data.csv'

In [11]:
spark.read.csv(PATH).show()

+--------------------+
|                 _c0|
+--------------------+
|month;country;cod...|
|01/2016;IT;620469...|
|01/2016;CN;900190...|
|01/2016;BY;841430...|
|01/2016;US;901850...|
|01/2016;EE;902110...|
|01/2016;FR;381600...|
|01/2016;MX;852351...|
|01/2016;JP;620452...|
|01/2016;KR;611020...|
|01/2016;KG;852713...|
|01/2016;ZA;842123...|
|01/2016;CN;851810...|
|01/2016;TR;841790...|
|01/2016;IT;390610...|
|01/2016;CZ;870840...|
|01/2016;ES;640419...|
|01/2016;IT;940490...|
|01/2016;UA;820780...|
|01/2016;CN;330410...|
+--------------------+
only showing top 20 rows



In [29]:
spark.read.csv(PATH, sep=';', header=True).show(10)

+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|  month|country|      code| value| netto|quantity|region|district|direction_eng|measure_eng|           load_date|
+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|01/2016|     IT|6204695000|   131|     1|       7| 46000|      01|           IM|        ShT|2024-07-01T00:00:...|
|01/2016|     CN|9001900009|112750|    18|       0| 46000|      01|           IM|          1|2024-01-01T00:00:...|
|01/2016|     BY|8414302004|   392|    57|       8| 50000|      06|           IM|        ShT|2024-06-01T00:00:...|
|01/2016|     US|9018509000| 54349|   179|       0| 40000|      02|           IM|          1|2024-04-01T00:00:...|
|01/2016|     EE|9021101000| 17304|   372|       0| 46000|      01|           IM|          1|2024-02-01T00:00:...|
|01/2016|     FR|3816000000|323488|253600|       0| 40000|      02|           IM

In [3]:
df = spark.read.csv(PATH, sep=';', header=True)


In [4]:
df.show(truncate=False)

+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+-----------------------------+
|month  |country|code      |value |netto |quantity|region|district|direction_eng|measure_eng|load_date                    |
+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+-----------------------------+
|01/2016|IT     |6204695000|131   |1     |7       |46000 |01      |IM           |ShT        |2024-07-01T00:00:00.000+03:00|
|01/2016|CN     |9001900009|112750|18    |0       |46000 |01      |IM           |1          |2024-01-01T00:00:00.000+03:00|
|01/2016|BY     |8414302004|392   |57    |8       |50000 |06      |IM           |ShT        |2024-06-01T00:00:00.000+03:00|
|01/2016|US     |9018509000|54349 |179   |0       |40000 |02      |IM           |1          |2024-04-01T00:00:00.000+03:00|
|01/2016|EE     |9021101000|17304 |372   |0       |46000 |01      |IM           |1          |2024-02-01T00:00:00.000+03:00|
|01/2016

In [76]:
df = df\
    .select('country', 'value', 'quantity', 'load_date')

In [6]:
data = [['KG', 123], ['UA', 234], ['CN', 345]]
df_d = spark.createDataFrame(data, ['name', 'id'])

In [10]:
new = df.join(df_d, df.country == df_d.name, 'inner')

In [11]:
new.explain()

== Physical Plan ==
*(5) SortMergeJoin [country#17], [name#150], Inner
:- *(2) Sort [country#17 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(country#17, 200), true, [id=#75]
:     +- *(1) Project [month#16, country#17, code#18, value#19, netto#20, quantity#21, region#22, district#23, direction_eng#24, measure_eng#25, load_date#26]
:        +- *(1) Filter isnotnull(country#17)
:           +- FileScan csv [month#16,country#17,code#18,value#19,netto#20,quantity#21,region#22,district#23,direction_eng#24,measure_eng#25,load_date#26] Batched: false, DataFilters: [isnotnull(country#17)], Format: CSV, Location: InMemoryFileIndex[file:/opt/workspace/data/customs_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country)], ReadSchema: struct<month:string,country:string,code:string,value:string,netto:string,quantity:string,region:s...
+- *(4) Sort [name#150 ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(name#150, 200), true, [id=#81]
      +- *(3) Filter isnotnu

In [12]:
new.show()

+-------+-------+----------+---------+-------+--------+------+--------+-------------+-----------+--------------------+----+---+
|  month|country|      code|    value|  netto|quantity|region|district|direction_eng|measure_eng|           load_date|name| id|
+-------+-------+----------+---------+-------+--------+------+--------+-------------+-----------+--------------------+----+---+
|01/2018|     UA|8484200000| 47127,34|      0|      45| 45000|      01|           IM|        ShT|2024-04-01T00:00:...|  UA|234|
|12/2020|     UA|3812390000| 50188,71|  27020|       0| 45000|      01|           IM|          1|2024-02-01T00:00:...|  UA|234|
|01/2018|     UA|8546200000| 29188,36|  33840|       0| 45000|      01|           IM|          1|2024-01-01T00:00:...|  UA|234|
|12/2020|     UA|3925901000|  5159,71|      0|       0| 45000|      01|           IM|          1|2024-03-01T00:00:...|  UA|234|
|01/2018|     UA|8606918000|914598,62|1224100|      45| 45000|      01|           EK|        ShT|2024-01

In [5]:
df.show()

+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|  month|country|      code| value| netto|quantity|region|district|direction_eng|measure_eng|           load_date|
+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|01/2016|     IT|6204695000|   131|     1|       7| 46000|      01|           IM|        ShT|2024-07-01T00:00:...|
|01/2016|     CN|9001900009|112750|    18|       0| 46000|      01|           IM|          1|2024-01-01T00:00:...|
|01/2016|     BY|8414302004|   392|    57|       8| 50000|      06|           IM|        ShT|2024-06-01T00:00:...|
|01/2016|     US|9018509000| 54349|   179|       0| 40000|      02|           IM|          1|2024-04-01T00:00:...|
|01/2016|     EE|9021101000| 17304|   372|       0| 46000|      01|           IM|          1|2024-02-01T00:00:...|
|01/2016|     FR|3816000000|323488|253600|       0| 40000|      02|           IM

In [60]:
df_new = spark\
    .read\
    .csv('data/no_control', header=True, sep=';')

In [86]:
df_new.show()

+-------+---------+--------+
|country|    value|quantity|
+-------+---------+--------+
|     AT| 46002,81|       0|
|     CN|204751,87|       0|
|     PL|   543,17|     200|
|     DE| 17448,94|       0|
|     BY| 234812,5|       0|
|     IT|118691,78|       0|
|     EE| 96092,76|       0|
|     KR| 77794,39|       0|
|     BY|    12650|       0|
|     KZ| 14664,06|       0|
|     BY|   567,87|       0|
|     LV|    58,74|       0|
|     DE| 90468,68|       0|
|     UA| 17613,74|       0|
|     TJ|226618,32|       0|
|     FI|  18519,2|       0|
|     DK| 61739,46|    2626|
|     CN|226175,44|       0|
|     DE|  17549,2|       0|
|     BE|  4711,56|       0|
+-------+---------+--------+
only showing top 20 rows



Сохранение таблицы без настроект


In [ ]:
df\
    .write\
    .format('csv')\
    .options(header=True, sep=';')\
    .csv('data/no_control')

Сохранение партицированной таблицы

In [ ]:
df\
    .write\
    .partitionBy('load_date')\
    .format('csv')\
    .options(header=True, sep=';')\
    .csv('data/partitionby')

Сохранение одним файлом

In [ ]:
df\
    .coalesce(1)\
    .write\
    .format('csv')\
    .options(header=True, sep=';')\
    .csv('data/one_file')

Сохранение партицированной таблицы одним файлом

In [95]:
df\
    .coalesce(1)\
    .write\
    .partitionBy('load_date')\
    .format('csv')\
    .options(header=True, sep=';')\
    .csv('data/one_file_with_part')

In [31]:
df = df.withColumnRenamed('direction_eng', 'direction')

In [57]:
result = df.withColumnRenamed('direction_eng', 'direction').withColumnRenamed('measure_eng', 'measure')

In [ ]:
result.withColumnsRenamed({'code':'new_code', 'country':'new_country'}).show()

In [58]:
df.show()

+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|  month|country|      code| value| netto|quantity|region|district|direction_eng|measure_eng|           load_date|
+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|01/2016|     IT|6204695000|   131|     1|       7| 46000|      01|           IM|        ShT|2024-07-01T00:00:...|
|01/2016|     CN|9001900009|112750|    18|       0| 46000|      01|           IM|          1|2024-01-01T00:00:...|
|01/2016|     BY|8414302004|   392|    57|       8| 50000|      06|           IM|        ShT|2024-06-01T00:00:...|
|01/2016|     US|9018509000| 54349|   179|       0| 40000|      02|           IM|          1|2024-04-01T00:00:...|
|01/2016|     EE|9021101000| 17304|   372|       0| 46000|      01|           IM|          1|2024-02-01T00:00:...|
|01/2016|     FR|3816000000|323488|253600|       0| 40000|      02|           IM

In [20]:
df2 = df.select('load_date', 'month', 'value')


df2 = df2.withColumn('load_date', F.col('load_date').cast('date'))

In [26]:
df = df.withColumn('load_date', df['load_date'].cast('date'))\
    .withColumn('region', df['region'].cast('int'))

### PrintSchema

In [17]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- quantity: integer (nullable = true)



In [7]:
result = df\
            .withColumnRenamed("direction_eng", "direction")\
            .withColumnRenamed("measure_eng", "measure")

result.columns

['month',
 'country',
 'code',
 'value',
 'netto',
 'quantity',
 'region',
 'district',
 'direction',
 'measure',
 'load_date']

### Select

In [59]:
result.select('country').distinct().show(10, truncate=False)

+-------+
|country|
+-------+
|LT     |
|MM     |
|DZ     |
|CI     |
|TC     |
|FI     |
|SC     |
|AZ     |
|UA     |
|RO     |
+-------+
only showing top 10 rows


### GroupBy

In [42]:
result\
    .groupBy('country')\
    .agg(F.count('*').alias('total_rows'))\
    .orderBy(F.col('total_rows').desc())\
    .show()

### Filter

In [52]:
result = result.withColumn('value', F.regexp_replace('value', ',', '.').cast('float')).show(5)

AttributeError: 'NoneType' object has no attribute 'withColumn'

In [51]:
result.printSchema()

AttributeError: 'NoneType' object has no attribute 'printSchema'

In [8]:
result\
    .where('''country == "DE"''')\
    .where('''value < 1000''')\
    .show()

+-------+-------+----------+-----+-----+--------+------+--------+---------+-------------+--------------------+
|  month|country|      code|value|netto|quantity|region|district|direction|      measure|           load_date|
+-------+-------+----------+-----+-----+--------+------+--------+---------+-------------+--------------------+
|01/2016|     DE|2918140000|  265|   15|       0| 45000|      01|       IM|            1|2024-03-01T00:00:...|
|01/2016|     DE|7306408008|  403|   16|       0| 70000|      01|       IM|            1|2024-01-01T00:00:...|
|01/2016|     DE|2106909803|  497|   66|       0| 40000|      02|       IM|            1|2024-04-01T00:00:...|
|01/2016|     DE|5906999000|  846|   60|       0| 45000|      01|       IM|            1|2024-01-01T00:00:...|
|01/2016|     DE|6112419000|   96|    2|      14| 45000|      01|       IM|          ShT|2024-01-01T00:00:...|
|01/2016|     DE|8407330000|  249|  160|       1| 50000|      06|       IM|          ShT|2024-02-01T00:00:...|
|

In [9]:
# Два варианта написании фильтрации
df_de = result\
            .where(F.col('country') == 'DE')\
            .where(F.col('value').isNotNull())

df_de2 = result\
            .where(''' country == "DE" ''')\
            .where(''' value IS NOT NULL ''')

print(df_de.count() == df_de2.count())

True


### Save to CSV

In [7]:
df.show(2, truncate=False)

+-------+-------+----------+------+-----+--------+------+--------+-------------+-----------+-----------------------------+
|month  |country|code      |value |netto|quantity|region|district|direction_eng|measure_eng|load_date                    |
+-------+-------+----------+------+-----+--------+------+--------+-------------+-----------+-----------------------------+
|01/2016|IT     |6204695000|131   |1    |7       |46000 |01      |IM           |ShT        |2024-07-01T00:00:00.000+03:00|
|01/2016|CN     |9001900009|112750|18   |0       |46000 |01      |IM           |1          |2024-01-01T00:00:00.000+03:00|
+-------+-------+----------+------+-----+--------+------+--------+-------------+-----------+-----------------------------+
only showing top 2 rows



In [15]:
df_de = df

In [75]:
df_de.columns

['country', 'value', 'quantity']

In [65]:
final = df_de\
            .select('month',
                    'country',
                    'code',
                    'value',
                    'netto',
                    'quantity',
                    'region',
                    'district',
                    'direction',
                    'measure',
                    F.col('load_date').cast('date'))

final.show(2, truncate=False)

+-------+-------+----------+-----+-----+--------+------+--------+---------+-------+----------+
|month  |country|code      |value|netto|quantity|region|district|direction|measure|load_date |
+-------+-------+----------+-----+-----+--------+------+--------+---------+-------+----------+
|01/2016|DE     |4016995709|5901 |172  |0       |46000 |01      |IM       |1      |2024-01-01|
|01/2016|DE     |8708809109|1213 |94   |0       |45000 |01      |IM       |1      |2024-01-01|
+-------+-------+----------+-----+-----+--------+------+--------+---------+-------+----------+
only showing top 2 rows


In [75]:
type(final)

pyspark.sql.classic.dataframe.DataFrame

In [18]:
final = df

In [10]:
final.coalesce(1).show(10)

+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|  month|country|      code| value| netto|quantity|region|district|direction_eng|measure_eng|           load_date|
+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|01/2016|     IT|6204695000|   131|     1|       7| 46000|      01|           IM|        ShT|2024-07-01T00:00:...|
|01/2016|     CN|9001900009|112750|    18|       0| 46000|      01|           IM|          1|2024-01-01T00:00:...|
|01/2016|     BY|8414302004|   392|    57|       8| 50000|      06|           IM|        ShT|2024-06-01T00:00:...|
|01/2016|     US|9018509000| 54349|   179|       0| 40000|      02|           IM|          1|2024-04-01T00:00:...|
|01/2016|     EE|9021101000| 17304|   372|       0| 46000|      01|           IM|          1|2024-02-01T00:00:...|
|01/2016|     FR|3816000000|323488|253600|       0| 40000|      02|           IM

In [27]:
final.show()

+-------+------+--------+
|country| value|quantity|
+-------+------+--------+
|     IT|   131|       7|
|     CN|112750|       0|
|     BY|   392|       8|
|     US| 54349|       0|
|     EE| 17304|       0|
|     FR|323488|       0|
|     MX|  1611|       4|
|     JP|    29|       2|
|     KR|   815|       5|
|     KG| 11868|    2630|
|     ZA| 12686|    3451|
|     CN|    12|      10|
|     TR|206453|       1|
|     IT|  4492|       0|
|     CZ|    41|       0|
|     ES| 11822|     760|
|     IT|  6801|       0|
|     UA| 35793|       0|
|     CN| 59678|       0|
|     SI|  1470|      15|
+-------+------+--------+
only showing top 20 rows



In [33]:
# Сохранение неконтроллируемое по кол-ву файлов
# df\
#     .write\
#     .format('csv')\
#     .options(header='True', sep=';')\
#     .csv('data/new_data')

# partition_num = final.rdd.getNumPartitions()
# print(f'Кол-во партиций {partition_num}')

# # Сохранение контроллируемое по кол-ву файлов - ОДИН ФАЙЛ
# final\
#     .coalesce(1)\
#     .write\
#     .format('csv')\
#     .options(header='True', sep=';')\
#     .csv('data/final_one_file') 

# partition_num = final.coalesce(1).rdd.getNumPartitions()
# print(f'Кол-во партиций {partition_num}')


# # Сохранения с партицированием
# final\
#     .coalesce(1) \
#     .write\
#     .partitionBy('country')\
#     .format('csv')\
#     .options(header='True', sep=';')\
#     .csv('data/final_partitioned')

# print_df = final.select('country').distinct()
# print(f'country distinct: {print_df.count()}')


# # Сохранения с партицированием и repartition внутри самой партиции
final\
    co\
    .write\
    .partitionBy('country')\
    .format('csv')\
    .options(header='True', sep=';')\
    .csv('data/final_partitioned_repart')

partition_num = final.repartition(1, 'country').rdd.getNumPartitions()
print(f'Кол-во партиций {partition_num}')

Кол-во партиций 1


In [41]:
new_df = spark \
    .read\
    .csv('data/final_one_file/', header=True, sep=';')\
    .groupBy('country')\
    .agg(F.count('*').alias('schet'))\
    .orderBy(F.col('schet').desc())

In [42]:
new_df.show()

+-------+-----+
|country|schet|
+-------+-----+
|     DZ|    1|
|     MM|    1|
|     LT|    1|
|     TC|    1|
|     CI|    1|
|     SC|    1|
|     AZ|    1|
|     FI|    1|
|     UA|    1|
|     KI|    1|
|     ZM|    1|
|     RO|    1|
|     SL|    1|
|     SB|    1|
|     LA|    1|
|     NL|    1|
|     BW|    1|
|     BS|    1|
|     MN|    1|
|     RE|    1|
+-------+-----+
only showing top 20 rows



### Read Transformed

In [99]:
reader_no_control = spark\
                        .read\
                        .csv('data/no_control/', header=True, sep=';')\
                        .where(''' country = 'GG' ''')

reader_final_one_file = spark\
                            .read\
                            .csv('data/one_file/', header=True, sep=';')\
                            .where(''' country = 'GG' ''')

reader_partitioned = spark\
                        .read\
                        .csv('data/partitionby', header=True, sep=';')\
                        .where(''' country = 'GG' ''')

reader_partitioned_repart = spark\
                                .read\
                                .csv('data/one_file_with_part', header=True, sep=';')\
                                .where(''' country = 'GG' ''')


reader_no_control.count() # number of files read: 16 | size of files read: 1064.7 MiB | 30.5 s (1.8 s, 2.0 s, 2.7 s )

reader_final_one_file.count() # number of files read: 1 | size of files read: 1064.7 MiB | 30.7 s (836 ms, 3.8 s, 3.9 s )

reader_partitioned.count() # number of files read: 160 | size of files read: 309.6 MiB | 20.7 s (916 ms, 3.1 s, 3.9 s )

reader_partitioned_repart.count() # number of files read: 10 | size of files read: 309.6 MiB | 16.6 s (385 ms, 2.1 s, 2.4 s )

9

### JOIN

In [100]:
data = [
    (14000, "Северный"),
    (11000, "Южный"),
    (10000, "Восточный"),
    (26000, "Западный"),
    (56000, "Центральный")
]

region_df = spark.createDataFrame(data, schema='region_id long, name string')

region_df.show()


customs_data = spark\
                .read\
                .csv('data/customs_data.csv', header=True, sep=';')

customs_data.show(2)

+---------+-----------+
|region_id|       name|
+---------+-----------+
|    14000|   Северный|
|    11000|      Южный|
|    10000|  Восточный|
|    26000|   Западный|
|    56000|Центральный|
+---------+-----------+

+-------+-------+----------+------+-----+--------+------+--------+-------------+-----------+--------------------+
|  month|country|      code| value|netto|quantity|region|district|direction_eng|measure_eng|           load_date|
+-------+-------+----------+------+-----+--------+------+--------+-------------+-----------+--------------------+
|01/2016|     IT|6204695000|   131|    1|       7| 46000|      01|           IM|        ShT|2024-07-01T00:00:...|
|01/2016|     CN|9001900009|112750|   18|       0| 46000|      01|           IM|          1|2024-01-01T00:00:...|
+-------+-------+----------+------+-----+--------+------+--------+-------------+-----------+--------------------+
only showing top 2 rows



In [102]:
# Отключим автоматический Broadcast JOIN
import time
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [108]:
# Замерим выполнение запроса без broadcast join

start_time = time.time()

customs_data.join(region_df, customs_data.region==region_df.region_id, "left").count()

end_time = time.time()

print(f"Elapsed time for join operation: {end_time - start_time:.2f} seconds")

Elapsed time for join operation: 9.40 seconds


In [104]:
# Замерим выполнение запроса c broadcast join

start_time = time.time()

customs_data.join(F.broadcast(region_df), customs_data.region == region_df.region_id, "left").count()

end_time = time.time()

print(f"Elapsed time for broadcast join operation: {end_time - start_time:.2f} seconds")

Elapsed time for broadcast join operation: 6.89 seconds


### Cache | Persist

In [110]:
customs_data.cache().count()

26392290

In [113]:
customs_data.unpersist()

DataFrame[month: string, country: string, code: string, value: string, netto: string, quantity: string, region: string, district: string, direction_eng: string, measure_eng: string, load_date: string]

In [54]:
from pyspark.storagelevel import StorageLevel

customs_data.persist(StorageLevel.DISK_ONLY).count()

### Repartition & Coalesce

In [114]:
data = [(1,'one'), (2,'two'), (3,'three'), (4,'four'),
        (5,'five'), (6,'six'), (7, 'seven'), (8, 'eight'),
        (9, 'nine')]

df = spark.createDataFrame(data, ['id', 'number'])

df.show()

+---+------+
| id|number|
+---+------+
|  1|   one|
|  2|   two|
|  3| three|
|  4|  four|
|  5|  five|
|  6|   six|
|  7| seven|
|  8| eight|
|  9|  nine|
+---+------+



In [115]:
# Намеренно перемешаем и поделим на 8 разделов
mix = df.repartition(8)
mix.rdd.glom().collect()

[[],
 [],
 [],
 [],
 [],
 [],
 [Row(id=1, number='one'),
  Row(id=5, number='five'),
  Row(id=6, number='six'),
  Row(id=8, number='eight'),
  Row(id=4, number='four'),
  Row(id=2, number='two'),
  Row(id=7, number='seven'),
  Row(id=3, number='three')],
 [Row(id=9, number='nine')]]

In [58]:
mix.repartition(3).rdd.glom().collect()

In [59]:
mix.coalesce(3).rdd.glom().collect()

In [116]:
mix.toPandas().head()

ImportError: Pandas >= 0.23.2 must be installed; however, it was not found.

In [ ]:
# OUT OF MEMORY

d = spark.read.csv('data/customs_data.csv', header=True, sep='\t')
d.collect()

In [34]:
spark.stop()

In [ ]:
'hello'